# Financial Optimization and Asset Management
## Markowitz model with side constraints
## Group 9

### Exercise 2

Step 1. We need to prepare the appropriate envirenment by importing and/or installing requried libraries.  

In [3]:
#pip install gurobipy
#pip install openpyxl  # to open excel file, delete hashtag if not installed

In [4]:
import gurobipy as gb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import portfolio_optimization_toolbox as pot

In [6]:
# load the data
df=pd.read_excel("/workspaces/FOAM/data.xlsx", sheet_name="Returns S&P Mib 30 ", index_col=0, parse_dates=True)

In [7]:
# we can drop the index name, and change the date format appropriately
df.index.name=None
df.index = df.index.strftime('%b-%Y')
df.head(3)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
Jan-2004,0.056941,0.055085,-0.005061,-0.001376,0.038502,0.004092,-0.003996,-0.023231,0.036722,-0.018517,0.023003,0.008654,-0.025495,-0.028095,0.134228,-0.035847,0.039152,-0.031417,0.027356,-0.027555
Feb-2004,-0.025385,-0.097992,-0.083576,-0.046853,-0.041082,0.020377,-0.064916,-0.025405,-0.035421,-0.140655,-0.066209,-0.094376,-0.037007,-0.009937,0.064431,-0.106229,-0.009411,-0.077847,0.004931,-0.083825
Mar-2004,0.017945,0.046305,0.010309,0.060723,0.029258,0.039441,0.069033,0.016084,0.029271,0.083825,0.039465,0.093684,0.059825,0.138686,-0.034589,0.051324,0.005409,-0.040125,0.049068,0.003866


In [8]:
# since the stocks are given in the range from A1 to A20, we prepared dictionary to replace them with their respective names
stocks = [
    "AL IM Equity", "AGL IM Equity", "AUTO IM Equity", "NTV IM Equity",
    "BFI IM Equity", "BIN IM Equity", "BPM IM Equity", "BPVN IM Equity",
    "BNL IM Equity", "BPU IM Equity", "BUL IM Equity", "CAP IM Equity",
    "EDN IM Equity", "ENEL IM Equity", "ENI IM Equity", "FWB IM Equity",
    "F IM Equity", "FNC IM Equity", "G IM Equity", "ES IM Equity"
]

stock_dict = {f"A{i+1}": stock.replace(" IM Equity", "") for i, stock in enumerate(stocks)}

In [9]:
# now we can replace them
df = df.rename(columns=stock_dict)
df.head(2)

,AL,AGL,AUTO,NTV,BFI,BIN,BPM,BPVN,BNL,BPU,BUL,CAP,EDN,ENEL,ENI,FWB,F,FNC,G,ES
Jan-2004,0.056941,0.055085,-0.005061,-0.001376,0.038502,0.004092,-0.003996,-0.023231,0.036722,-0.018517,0.023003,0.008654,-0.025495,-0.028095,0.134228,-0.035847,0.039152,-0.031417,0.027356,-0.027555
Feb-2004,-0.025385,-0.097992,-0.083576,-0.046853,-0.041082,0.020377,-0.064916,-0.025405,-0.035421,-0.140655,-0.066209,-0.094376,-0.037007,-0.009937,0.064431,-0.106229,-0.009411,-0.077847,0.004931,-0.083825
